# Cartographie de la culture du riz au Bénin dans certaines localités

### Importation des biblios nécessaires


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pp
from datetime import datetime
from pcse.fileinput import CABOFileReader, YAMLCropDataProvider
from pathlib import Path
from nevergrad.functions.irrigation.common_path import IRRIGATION_DATA_DIR
from pcse.util import WOFOST71SiteDataProvider
from pcse.base import ParameterProvider
from pcse.db import NASAPowerWeatherDataProvider
from nevergrad.functions.irrigation.irrigation import KNOWN_GEOLOCS
from pcse.fileinput import YAMLAgroManagementReader
import yaml
from pcse.models import Wofost72_WLP_FD

### Mise en place du simulateur

In [ ]:
def model(wdp):
    """ Cette fonction va nous permettre de mettre en place le simulateur"""
    #mise en place des paramètres du modèles
    #paramètre de culture
    cropfile = "https://raw.githubusercontent.com/ajwdewit/WOFOST_crop_parameters/master/"
    cropdata = YAMLCropDataProvider(repository=cropfile)
    pp(cropdata.get_crops_varieties())
    
    #paramètre de sol
    soilfile = Path(IRRIGATION_DATA_DIR, "soil", "ec3.soil") 
    # we will use the water balance for 
    # freely draining soils and use the soil file for medium fine sand
    soildata = CABOFileReader(soilfile)
    pp(soildata)
    
    #paramètre de site
    sitedata = WOFOST71SiteDataProvider(
    WAV=100, # initial soil moisture content
    CO2=360, # the atmospheric CO2 concentration 
    )
    expected_sitedata = {
    'IFUNRN': 0,
    'NOTINF': 0,
    'SSI': 0.0, # the initial surface storage
    'SSMAX': 0.0, # the maximum surface storage
    'WAV': 100.0,
    'SMLIM': 0.4
     }
    assert sitedata == expected_sitedata
    
    #combinaison des trois premiers params
    parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)
    
    #paramètre de météo
    wdp=wdp
    crop_start_date = datetime.strptime("2006-03-31", "%Y-%m-%d").date()
    crop_end_date = datetime.strptime("2006-10-20", "%Y-%m-%d").date()
    df_weatherdataprovider = pd.DataFrame(wdp.export()).set_index("DAY")
    mask = (df_weatherdataprovider.index >= crop_start_date) & (df_weatherdataprovider.index <= crop_end_date)
    #df_weatherdataprovider.loc[mask, ["RAIN"]].plot()
    
    #paramètre d'agrogestion
    def read_yaml_agro(yaml_agro_file: Path) -> list:
        assert yaml_agro_file.exists()
        with open(yaml_agro_file, "r") as f:
            yaml_agro = f.read()
        return yaml.safe_load(yaml_agro)
    
    yaml_agro_file = Path(IRRIGATION_DATA_DIR, "agromanagement", "rice_schedule.yaml")
    agromanagement = read_yaml_agro(yaml_agro_file)
    
    #Initialisation du modèle
    wofost_sim = Wofost72_WLP_FD(parameters, wdp, agromanagement)
    #Execution du modèle
    wofost_sim.run_till_terminate()
    output = wofost_sim.get_output()
    df_output = pd.DataFrame(output)
    return df_output["LAI"].sum()


### Construction de la liste des villes et leurs coordonnées géographiques


In [ ]:
towns_localisations=[(6.664522047548846,2.159156765243099),
      (6.371274272103359,2.3961754953536243),
     (6.515052473722593, 2.5889961931913628),
      (6.983211693487492,2.663412046468264),
      (6.805119466540308,1.7876815558461814),
      (6.644998836178613,1.7178581535119701),
      (7.9741525617502225,2.2449016366623584),
      (7.7850247569512945,2.1991596742641524),
      (11.863624885975188,3.3846903257980436),
      (11.13057733827704,2.9321041548916362),
      (8.899825860624906,2.5778173128877717),
      (9.353878406271505,2.5793832030491433),
      (11.248665869642261,1.5019846363736715),
      (10.298391303599118,1.3786018564760418)
      ]
towns=["Allada","Cotonou","Porto-Novo","Pobè","Dogbo","Lokossa","Glazoué","Dassa-Zoumè","Malanville","Kandi","Tchaourou","Parakou","Pendjari","Natitingou"]

### Phase d'éxecution du modèle sur les différentes localités

In [ ]:

for i in range(len(towns)):
    ville = towns[i]
    ville_lat,ville_long =towns_localisations[i]
    print("lieu_simu: ",ville)
    wdp = NASAPowerWeatherDataProvider(
    latitude=ville_lat, longitude=ville_long,)
    simu = model(wdp)
    print("La valeur du LAI dans la " f"ville: {ville}" " est:",simu)
    a
    

In [ ]:
Dict_of_LAI={"Allada":"66.30768901417025",
             "Cotonou":"99.53093316197644",
             "Porto-Novo":"99.6067895038238",
             "Pobè":"91.45671393074487",
             "Dogbo":"63.86755379601517",
             "Lokossa":"43.36750654174513",
             "Glazoué":"28.168142832413395",
             "Dassa-Zoumè":"28.153579030413333",
             "Malanville":"5.458992118422631",
             "Kandi":"12.24171244184637",
             "Tchaourou":"24.46508016166584",
             "Parakou":"23.434981291124092",
             "Pendjari":"15.384599622627363",
             "Natitingou":"23.632902833901834"}